# Load a single ROI from an OME-Zarr file
To enable 3D interactivity and easy ways to interact with the dataset in a jupyter notebook, let's load a single ROI instead of the whole well.  
If you did not create your own OME-Zarr file using Fractal, you can download the output from here:
https://zenodo.org/record/7120354

In [ ]:
# In case anndata & matplotlib were not installed yet, run this
# pip install anndata
# pip install matplotlib

In [ ]:
import napari
import zarr
import ome_zarr
from pathlib import Path
import anndata as ad
import dask.array as da
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from utils_roi_loading import convert_ROI_table_to_indices, load_label_roi, load_intensity_roi, prepare_feature_vis

In [ ]:
# Define the path to your OME-Zarr file
# (either just processed via Fractal or downloaded from Zenodo)
zarr_url = Path('/Users/joel/Desktop/20220928_2x2_Zenodo/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr')

In [ ]:
# Constant settings
labels_name = 'label_DAPI'
roi_table = 'FOV_ROI_table'

# Changing settings
well = 'B/03'
roi_index_of_interest = 0
channel_index = 0

In [ ]:
# Look at the ROI table
roi_an = ad.read_zarr(zarr_url / f'{well}/0/tables/{roi_table}')
ro_df = roi_an.to_df()
ro_df

In [ ]:
label_roi, scale_label = load_label_roi(zarr_url, well, roi_index_of_interest, labels_name, level = 0)
img_roi, scale_img = load_intensity_roi(zarr_url, well, roi_index_of_interest, channel_index, level = 0)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(img_roi, scale=scale_img)
label_layer = viewer.add_labels(label_roi, scale=scale_label)

# Feature visualization

In [ ]:
measurement_table = 'nuclei'
nuclei_path = zarr_url / f'{well}/0/tables/{measurement_table}'
nuclei_an = ad.read_zarr(nuclei_path)

In [ ]:
feature_df = nuclei_an.to_df()
feature_df[['area', 'mean_intensity', 'standard_deviation_intensity']]

In [ ]:
colors, label_properties = prepare_feature_vis(feature_df, feature='area')
label_layer.properties = label_properties
label_layer.color = colors

# 3D example

The same ROI loading & feature visualization also works on 3D data. Because 3D segmentation & feature measurements are slower, we didn't run them in today's demo. But you can get the above dataset with 3D segmentation & 3D measurements here: https://zenodo.org/record/7144919

In [ ]:
zarr_url = Path('/Users/joel/Desktop/20221004_Zenodo_3DSeg/20200812-CardiomyocyteDifferentiation14-Cycle1.zarr')

In [ ]:
# Constant settings
labels_name = 'label_DAPI'
roi_table = 'FOV_ROI_table'

# Changing settings
well = 'B/03'
roi_index_of_interest = 0
channel_index = 0

In [ ]:
label_roi, scale_label = load_label_roi(zarr_url, well, roi_index_of_interest, labels_name, level = 0)
img_roi, scale_img = load_intensity_roi(zarr_url, well, roi_index_of_interest, channel_index, level = 0)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(img_roi, scale=scale_img)
label_layer = viewer.add_labels(label_roi, scale=scale_label)

In [ ]:
measurement_table = 'nuclei_3D'
nuclei_path = zarr_url / f'{well}/0/tables/{measurement_table}'
feature_df = ad.read_zarr(nuclei_path).to_df()
colors, label_properties = prepare_feature_vis(feature_df, feature='area')
label_layer.properties = label_properties
label_layer.color = colors